<a href="https://colab.research.google.com/github/NinaNikolova/data_mining_project/blob/main/python_games4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Random Forest модела за прогнозиране на потребителски рейтинг на видео игри

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()

Mounted at /content/drive


Saving video_game_reviews.csv to video_game_reviews.csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import joblib
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Сваляне на необходимите NLTK ресурси
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Конфигуриране на визуализациите
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
class VideoGameRandomForestModel:
    """
    Модел с Random Forest за прогнозиране на потребителски рейтинг на видео игри
    """

    def __init__(self, random_state=42):
        self.random_state = random_state
        self.model = None
        self.preprocessor = None
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def load_data(self, filepath):
        """Зареждане на данните от CSV файл"""
        print("Зареждане на данните...")
        df = pd.read_csv('/content/video_game_reviews.csv')
        print(df.head())
        print(f"Заредени са {df.shape[0]} реда и {df.shape[1]} колони")
        return df

    def explore_data(self, df):
        """Първоначално изследване на данните"""
        print("\n===== Изследване на данните =====")
        print(f"Размер на данните: {df.shape}")

        print("\nПървите 5 реда:")
        print(df.head())

        print("\nИнформация за данните:")
        print(df.info())

        print("\nОсновни статистики:")
        print(df.describe(include='all'))

        print("\nЛипсващи стойности:")
        missing_values = df.isnull().sum()
        print(missing_values[missing_values > 0])

        # Визуализация на целевата променлива
        if 'User Rating' in df.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(df['User Rating'], kde=True)
            plt.title('Разпределение на потребителските оценки')
            plt.xlabel('Потребителска оценка')
            plt.ylabel('Брой')
            plt.savefig('user_rating_distribution.png')
            plt.close()
            print("Запазена е визуализация на разпределението на потребителските оценки")

        return df

    def clean_text(self, text):
        """Почистване и предварителна обработка на текстови данни"""
        if isinstance(text, str):
            # Конвертиране в малки букви
            text = text.lower()

            # Премахване на HTML тагове
            text = re.sub(r'<.*?>', '', text)

            # Премахване на специални символи и числа
            text = re.sub(r'[^a-zA-Z\s]', '', text)

            # Токенизация
            tokens = word_tokenize(text)

            # Премахване на стоп думи и лематизация
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]

            return ' '.join(tokens)
        else:
            return ''

    def preprocess_data(self, df):
        """Предварителна обработка на данните"""
        print("\n===== Предварителна обработка на данните =====")

        # Копиране на оригиналните данни
        df_processed = df.copy()

        # Почистване на текстови полета, ако съществуват
        text_columns = ['review_text', 'summary', 'description']
        for col in text_columns:
            if col in df_processed.columns:
                print(f"Почистване на текста в колона {col}...")
                df_processed[f'{col}_cleaned'] = df_processed[col].apply(self.clean_text)

        # Извличане на характеристики от датите, ако съществуват
        date_columns = ['release_date', 'review_date']
        for col in date_columns:
            if col in df_processed.columns:
                try:
                    df_processed[col] = pd.to_datetime(df_processed[col])
                    df_processed[f'{col}_year'] = df_processed[col].dt.year
                    df_processed[f'{col}_month'] = df_processed[col].dt.month
                    print(f"Извлечени са характеристики от колона {col}")
                except:
                    print(f"Не може да се конвертира {col} към datetime формат")

        # Премахване на редове с липсващи целеви стойности
        if 'user_rating' in df_processed.columns and df_processed['user_rating'].isnull().any():
            missing_count = df_processed['user_rating'].isnull().sum()
            print(f"Премахване на {missing_count} реда с липсващи целеви стойности")
            df_processed = df_processed.dropna(subset=['user_rating'])

        return df_processed

    def feature_engineering(self, df):
        """Създаване на нови характеристики от съществуващите данни"""
        print("\n===== Създаване на нови характеристики =====")

        # Копиране на данните
        df_featured = df.copy()

        # Създаване на характеристики на базата на дължината на текста, ако са налични
        text_columns = ['review_text_cleaned', 'summary_cleaned', 'description_cleaned']
        for col in text_columns:
            if col in df_featured.columns:
                df_featured[f'{col}_length'] = df_featured[col].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
                print(f"Създадена е характеристика за дължина на текста за {col}")

        # Създаване на характеристики на базата на разлики в оценките
        if 'critic_rating' in df_featured.columns and 'user_rating' in df_featured.columns:
            df_featured['user_critic_diff'] = df_featured['user_rating'] - df_featured['critic_rating']
            print("Създадена е характеристика за разлика между потребителска и критична оценка")

        # Създаване на характеристики, базирани на жанра
        if 'genre' in df_featured.columns:
            try:
                # Проверка на формата на жанровете
                if isinstance(df_featured['genre'].iloc[0], str) and ('[' in df_featured['genre'].iloc[0] or ',' in df_featured['genre'].iloc[0]):
                    # Конвертиране на текстови списъци в истински списъци
                    df_featured['genre'] = df_featured['genre'].apply(
                        lambda x: str(x).replace('[', '').replace(']', '').replace("'", "").split(',') if pd.notna(x) else []
                    )

                # Извличане на уникалните жанрове
                all_genres = set()
                for genres in df_featured['genre']:
                    if isinstance(genres, list):
                        all_genres.update([g.strip() for g in genres if g.strip()])
                    elif pd.notna(genres):
                        all_genres.add(str(genres).strip())

                # Създаване на бинарни характеристики за всеки жанр
                for genre in all_genres:
                    genre_name = genre.strip()
                    if genre_name:
                        df_featured[f'genre_{genre_name}'] = df_featured['genre'].apply(
                            lambda x: 1 if isinstance(x, list) and genre_name in [g.strip() for g in x] else
                                      (1 if str(x).strip() == genre_name else 0)
                        )

                print(f"Създадени са бинарни характеристики за {len(all_genres)} жанра")
            except Exception as e:
                print(f"Грешка при създаване на характеристики за жанр: {e}")

        return df_featured

    def prepare_data_for_model(self, df, target_col='user_rating'):
        """Подготовка на данните за моделиране"""
        print("\n===== Подготовка на данните за моделиране =====")

        # Проверка дали целевата колона съществува
        if target_col not in df.columns:
            print(f"Грешка: Целевата колона '{target_col}' не е намерена в данните")
            return None, None, None, None

        # Дефиниране на характеристики и цел
        X = df.drop(columns=[target_col])
        y = df[target_col]

        # Разделяне на данните на обучителни и тестови
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=self.random_state
        )

        print(f"Размер на обучителните данни: {X_train.shape}")
        print(f"Размер на тестовите данни: {X_test.shape}")

        return X_train, X_test, y_train, y_test

    def build_preprocessor(self, X_train):
        """Изграждане на процес за предварителна обработка"""
        print("\n===== Изграждане на процес за предварителна обработка =====")

        # Идентифициране на типовете характеристики
        categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
        numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        text_cols = [col for col in X_train.columns if 'cleaned' in col and col in categorical_cols]

        # Премахване на текстовите колони от категорийните
        categorical_cols = [col for col in categorical_cols if col not in text_cols]

        # Игнориране на колоните с дати
        date_cols = [col for col in X_train.columns if 'date' in col and not ('year' in col or 'month' in col)]
        categorical_cols = [col for col in categorical_cols if col not in date_cols]
        numeric_cols = [col for col in numeric_cols if col not in date_cols]

        # Премахване на колони за игнориране
        cols_to_ignore = ['id', 'title', 'review_text', 'summary', 'description', 'genre', 'platform']
        categorical_cols = [col for col in categorical_cols if col not in cols_to_ignore]

        # Създаване на трансформатор за колоните
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([
                    ('imputer', SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler())
                ]), numeric_cols),
                ('cat', Pipeline([
                    ('imputer', SimpleImputer(strategy='most_frequent')),
                    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
                ]), categorical_cols)
            ],
            remainder='drop'  # Игнориране на неизползваните колони
        )

        # Добавяне на текстови характеристики, ако има такива
        if text_cols:
            for i, text_col in enumerate(text_cols):
                transformer_name = f'text_{i}'
                text_transformer = Pipeline([
                    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
                    ('tfidf', TfidfVectorizer(max_features=1000, min_df=5, max_df=0.8))
                ])
                preprocessor = ColumnTransformer(
                    transformers=preprocessor.transformers + [(transformer_name, text_transformer, [text_col])],
                    remainder=preprocessor.remainder
                )

        print(f"Численични колони за обработка: {numeric_cols}")
        print(f"Категорийни колони за обработка: {categorical_cols}")
        print(f"Текстови колони за обработка: {text_cols}")

        self.preprocessor = preprocessor
        return preprocessor

    def train_random_forest(self, X_train, y_train, X_test, y_test):
        """Обучение на модел Random Forest с оптимизация на хиперпараметрите"""
        print("\n===== Обучение на Random Forest модел =====")

        # Създаване на препроцесор, ако няма наличен
        if self.preprocessor is None:
            self.build_preprocessor(X_train)

        # Дефиниране на основния Random Forest модел
        rf = RandomForestRegressor(random_state=self.random_state)

        # Дефиниране на пълната тръба за обработка
        pipeline = Pipeline([
            ('preprocessor', self.preprocessor),
            ('model', rf)
        ])

        # Дефиниране на параметри за търсене
        param_grid = {
            'model__n_estimators': [50, 100, 200],
            'model__max_depth': [None, 10, 20, 30],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['auto', 'sqrt']
        }

        # Използване на RandomizedSearchCV за по-бързо търсене
        print("Започване на оптимизация на хиперпараметрите...")
        random_search = RandomizedSearchCV(
            pipeline,
            param_distributions=param_grid,
            n_iter=20,
            cv=3,
            scoring='neg_root_mean_squared_error',
            random_state=self.random_state,
            n_jobs=-1
        )

        # Обучение на модела
        random_search.fit(X_train, y_train)

        # Вземане на най-добрите параметри и модел
        best_params = random_search.best_params_
        print(f"Най-добри параметри: {best_params}")

        self.model = random_search.best_estimator_

        # Оценка на модела
        print("\n===== Оценка на модела =====")
        y_pred_train = self.model.predict(X_train)
        y_pred_test = self.model.predict(X_test)

        # Изчисляване на метрики върху обучителните данни
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        train_mae = mean_absolute_error(y_train, y_pred_train)
        train_r2 = r2_score(y_train, y_pred_train)

        # Изчисляване на метрики върху тестовите данни
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_r2 = r2_score(y_test, y_pred_test)

        print(f"Метрики върху обучителните данни:")
        print(f"  - RMSE: {train_rmse:.4f}")
        print(f"  - MAE: {train_mae:.4f}")
        print(f"  - R²: {train_r2:.4f}")

        print(f"Метрики върху тестовите данни:")
        print(f"  - RMSE: {test_rmse:.4f}")
        print(f"  - MAE: {test_mae:.4f}")
        print(f"  - R²: {test_r2:.4f}")

        # Визуализация на действителни спрямо прогнозни стойности
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred_test, alpha=0.7)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel('Действителна потребителска оценка')
        plt.ylabel('Прогнозна потребителска оценка')
        plt.title('Действителни спрямо прогнозни потребителски оценки')
        plt.savefig('actual_vs_predicted.png')
        plt.close()
        print("Запазена е визуализация на действителни спрямо прогнозни стойности")

        return {
            'model': self.model,
            'train_metrics': {
                'rmse': train_rmse,
                'mae': train_mae,
                'r2': train_r2
            },
            'test_metrics': {
                'rmse': test_rmse,
                'mae': test_mae,
                'r2': test_r2
            }
        }

    def feature_importance(self):
        """Анализ на важността на характеристиките"""
        print("\n===== Анализ на важността на характеристиките =====")

        if self.model is None:
            print("Модел не е обучен. Моля, обучете модел преди да анализирате важността на характеристиките.")
            return None

        # Извличане на важността на характеристиките
        rf_model = self.model.named_steps['model']
        preprocessor = self.model.named_steps['preprocessor']

        # Получаване на имената на характеристиките
        feature_names = []

        # Извличане на имената на характеристиките от всеки трансформатор
        for name, _, cols in preprocessor.transformers_:
            if name == 'remainder':
                continue

            # Специална обработка за TfidfVectorizer
            if 'text' in name:
                tfidf = preprocessor.named_transformers_[name].named_steps['tfidf']
                feature_names.extend([f"{cols[0]}_{word}" for word in tfidf.get_feature_names_out()])
            else:
                if hasattr(preprocessor.named_transformers_[name], 'get_feature_names_out'):
                    transformer_feature_names = preprocessor.named_transformers_[name].get_feature_names_out(cols)
                    feature_names.extend(transformer_feature_names)
                elif name == 'num':
                    feature_names.extend(cols)

        # Извличане на стойности за важност
        importances = rf_model.feature_importances_

        # Проверка дали съвпадат размерите
        if len(feature_names) == len(importances):
            feature_importance = pd.DataFrame({
                'Характеристика': feature_names,
                'Важност': importances
            })

            # Сортиране по важност
            feature_importance = feature_importance.sort_values('Важност', ascending=False)

            # Извеждане на топ 20 характеристики
            print("\nТоп 20 най-важни характеристики:")
            print(feature_importance.head(20))

            # Визуализация на важността на характеристиките
            plt.figure(figsize=(12, 10))
            sns.barplot(x='Важност', y='Характеристика', data=feature_importance.head(20))
            plt.title('Важност на характеристиките')
            plt.tight_layout()
            plt.savefig('feature_importance.png')
            plt.close()
            print("Запазена е визуализация на важността на характеристиките")

            return feature_importance
        else:
            print(f"Грешка: Несъответствие между имената на характеристиките ({len(feature_names)}) и стойностите за важност ({len(importances)})")
            return None

    def save_model(self, filepath):
        """Запазване на модела във файл"""
        if self.model is not None:
            joblib.dump(self.model, filepath)
            print(f"Моделът е запазен в {filepath}")
        else:
            print("Няма модел за запазване. Моля, обучете модел първо.")

    def run_pipeline(self, filepath, target_col='User Rating', save_model_path=None):
        """Изпълнение на пълната обработка"""
        # Зареждане и изследване на данните
        df = self.load_data(filepath)
        self.explore_data(df)

        # Предварителна обработка на данните
        df_processed = self.preprocess_data(df)

        # Създаване на нови характеристики
        df_featured = self.feature_engineering(df_processed)

        # Подготовка на данните за модела
        data_split = self.prepare_data_for_model(df_featured, target_col)
        if data_split[0] is None:
            return None

        X_train, X_test, y_train, y_test = data_split

        # Изграждане на процеса за предварителна обработка
        self.build_preprocessor(X_train)

        # Обучение на Random Forest модела
        results = self.train_random_forest(X_train, y_train, X_test, y_test)

        # Анализ на важността на характеристиките
        self.feature_importance()

        # Запазване на модела, ако е предоставен път
        if save_model_path:
            self.save_model(save_model_path)

        return results

# Пример за използване:
if __name__ == "__main__":
    # Създаване на модела
    model = VideoGameRandomForestModel()

    # Изпълнение на пълната обработка (заменете с вашия път до файла)
    filepath = "video_game_reviews.csv"  # Свален от Kaggle
    results = model.run_pipeline(filepath, save_model_path="video_game_rf_model.pkl")

Зареждане на данните...
           Game Title  User Rating Age Group Targeted  Price     Platform  \
0  Grand Theft Auto V         36.4           All Ages  41.41           PC   
1          The Sims 4         38.3             Adults  57.56           PC   
2           Minecraft         26.8              Teens  44.93           PC   
3   Bioshock Infinite         38.4           All Ages  48.29       Mobile   
4     Half-Life: Alyx         30.1             Adults  55.49  PlayStation   

  Requires Special Device   Developer        Publisher  Release Year  \
0                      No  Game Freak       Innersloth          2015   
1                      No    Nintendo  Electronic Arts          2015   
2                     Yes      Bungie           Capcom          2012   
3                     Yes  Game Freak         Nintendo          2015   
4                     Yes  Game Freak       Epic Games          2022   

       Genre Multiplayer  Game Length (Hours) Graphics Quality  \
0  Adventure  